In [1]:
from pathlib import Path
import numpy as np
import os
import pandas as pd
import re
import time

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon_EnetV2_OpenMax/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [3]:
import os
import pandas as pd
from collections import defaultdict

# Đường dẫn thư mục chứa các file attack
input_dir = '../data/set_04/test_03_known_vehicle_unknown_attack'
output_dir = '../data/set_04/test_03_known_vehicle_unknown_attack/merged_one/dos'
os.makedirs(output_dir, exist_ok=True)

# Danh sách các nhóm attack chính (không còn 'accessory' vì gộp vào 'attack-free')
# main_attack_types = ['dos', 'double', 'force-neutral', 'fuzzing', 'triple']
main_attack_types = ['dos']
# Gom file theo nhóm attack chính
grouped_files = defaultdict(list)

# Duyệt toàn bộ file
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        # Gộp accessory-3.csv và accessory-4.csv vào attack-free
        if file_name in ['accessory-3.csv', 'accessory-4.csv']:
            grouped_files['attack-free'].append(file_name)
            continue

        # Tìm attack type chính phù hợp với tên file
        matched = False
        for atk in main_attack_types:
            if atk in file_name.lower():
                grouped_files[atk].append(file_name)
                matched = True
                break
        if not matched:
            print(f"⚠️ Không xác định được nhóm cho file: {file_name}")

# Duyệt từng nhóm để merge
for group, file_list in grouped_files.items():
    df_list = []
    print(f"\n📂 Đang gộp nhóm: {group}")

    for file_name in file_list:
        file_path = os.path.join(input_dir, file_name)
        try:
            df = pd.read_csv(file_path)

            # Xử lý arbitration_id
            if 'arbitration_id' in df.columns:
                df['arbitration_id'] = df['arbitration_id'].apply(
                    lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
                )

            # Xử lý data_field
            if 'data_field' in df.columns:
                df['data_field'] = df['data_field'].apply(
                    lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
                )

            # Lấy cột chính nếu có
            keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
            df = df[[col for col in keep_cols if col in df.columns]].dropna()

            df_list.append(df)
        except Exception as e:
            print(f"[!] Lỗi đọc file {file_name}: {e}")

    # Gộp và lưu
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        output_file = os.path.join(output_dir, f'{group}.csv')
        merged_df.to_csv(output_file, index=False)
        print(f"✅ Đã lưu file: {output_file}")
    else:
        print(f"⚠️ Không có file hợp lệ trong nhóm {group}")


⚠️ Không xác định được nhóm cho file: double-2.csv
⚠️ Không xác định được nhóm cho file: fuzzing-2.csv
⚠️ Không xác định được nhóm cho file: force-neutral-1.csv
⚠️ Không xác định được nhóm cho file: triple-2.csv
⚠️ Không xác định được nhóm cho file: triple-1.csv
⚠️ Không xác định được nhóm cho file: double-1.csv
⚠️ Không xác định được nhóm cho file: force-neutral-2.csv
⚠️ Không xác định được nhóm cho file: fuzzing-1.csv

📂 Đang gộp nhóm: dos
✅ Đã lưu file: ../data/set_04/test_03_known_vehicle_unknown_attack/merged_one/dos/dos.csv


In [ ]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

# Đường dẫn thư mục chứa các file attack
input_dir = '../data/set_04/train_01'  # Thay bằng đường dẫn thật nếu cần
output_dir = '../data/set_04/train_01/merged'   # Thư mục đầu ra
os.makedirs(output_dir, exist_ok=True)

# Danh sách các nhóm attack chính (không còn 'accessory' vì gộp vào 'attack-free')
# main_attack_types = ['rpm', 'speed', 'interval', 'standstill', 'systematic', 'attack-free']
main_attack_types = ['rpm', 'speed', 'interval', 'standstill', 'systematic', 'attack-free']

# Gom file theo nhóm attack chính
grouped_files = defaultdict(list)

# Duyệt toàn bộ file
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        # Gộp accessory-3.csv và accessory-4.csv vào attack-free
        if file_name in ['accessory-3.csv', 'accessory-4.csv']:
            grouped_files['attack-free'].append(file_name)
            continue

        # Tìm attack type chính phù hợp với tên file
        matched = False
        for atk in main_attack_types:
            if atk in file_name.lower():
                grouped_files[atk].append(file_name)
                matched = True
                break
        if not matched:
            print(f"⚠️ Không xác định được nhóm cho file: {file_name}")

# Duyệt từng nhóm để merge
for group, file_list in grouped_files.items():
    df_list = []
    print(f"\n📂 Đang gộp nhóm: {group}")

    for file_name in file_list:
        file_path = os.path.join(input_dir, file_name)
        try:
            df = pd.read_csv(file_path)

            # Xử lý arbitration_id
            if 'arbitration_id' in df.columns:
                df['arbitration_id'] = df['arbitration_id'].apply(
                    lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
                )

            # Xử lý data_field
            if 'data_field' in df.columns:
                df['data_field'] = df['data_field'].apply(
                    lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
                )

            # Lấy cột chính nếu có
            keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
            df = df[[col for col in keep_cols if col in df.columns]].dropna()

            df_list.append(df)
        except Exception as e:
            print(f"[!] Lỗi đọc file {file_name}: {e}")

    # Gộp và lưu cho các nhóm khác ngoài 'attack-free'
    if group != 'attack-free':
        if df_list:
            merged_df = pd.concat(df_list, ignore_index=True)
            output_file = os.path.join(output_dir, f'{group}.csv')
            merged_df.to_csv(output_file, index=False)
            print(f"✅ Đã lưu file: {output_file}")
        else:
            print(f"⚠️ Không có file hợp lệ trong nhóm {group}")

# Chia dữ liệu của 'attack-free' vào các nhóm attack khác
attack_free_df_list = []

# Đọc tất cả các tệp 'attack-free'
for file_name in grouped_files['attack-free']:
    file_path = os.path.join(input_dir, file_name)
    try:
        df = pd.read_csv(file_path)

        # Xử lý dữ liệu tương tự như trên
        if 'arbitration_id' in df.columns:
            df['arbitration_id'] = df['arbitration_id'].apply(
                lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
            )

        if 'data_field' in df.columns:
            df['data_field'] = df['data_field'].apply(
                lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
            )

        keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
        df = df[[col for col in keep_cols if col in df.columns]].dropna()

        attack_free_df_list.append(df)
    except Exception as e:
        print(f"[!] Lỗi đọc file {file_name}: {e}")

# Gộp tất cả dữ liệu 'attack-free'
attack_free_df = pd.concat(attack_free_df_list, ignore_index=True)

# Chia đều dữ liệu 'attack-free' vào các nhóm attack chính
attack_free_samples_per_group = len(attack_free_df) // len(main_attack_types[:-1])  # Không tính 'attack-free' ở đây
attack_free_split = np.array_split(attack_free_df, len(main_attack_types)-1)

# Lưu kết quả và ghi đè lên các file tấn công hiện tại
for idx, group in enumerate(main_attack_types[:-1]):  # Bỏ qua 'attack-free'
    group_file_path = os.path.join(output_dir, f'{group}.csv')
    attack_free_data = attack_free_split[idx]
    
    # Đọc dữ liệu nhóm tấn công hiện tại
    group_data = pd.read_csv(group_file_path)
    
    # Gộp dữ liệu nhóm hiện tại với dữ liệu 'attack-free' chia đều
    merged_data = pd.concat([group_data, attack_free_data], ignore_index=True)
    
    # Ghi đè lên file tấn công hiện tại
    merged_data.to_csv(group_file_path, index=False)
    print(f"✅ Đã ghi đè file: {group_file_path}")

print("DONE!")



📂 Đang gộp nhóm: rpm
✅ Đã lưu file: ../data/set_04/train_01/merged/rpm.csv

📂 Đang gộp nhóm: attack-free

📂 Đang gộp nhóm: standstill
✅ Đã lưu file: ../data/set_04/train_01/merged/standstill.csv

📂 Đang gộp nhóm: speed
✅ Đã lưu file: ../data/set_04/train_01/merged/speed.csv

📂 Đang gộp nhóm: systematic
✅ Đã lưu file: ../data/set_04/train_01/merged/systematic.csv

📂 Đang gộp nhóm: interval
✅ Đã lưu file: ../data/set_04/train_01/merged/interval.csv


/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


✅ Đã ghi đè file: ../data/set_04/train_01/merged/rpm.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged/speed.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged/interval.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged/standstill.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged/systematic.csv
DONE!


## No Accessory

In [3]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

# Đường dẫn thư mục chứa các file attack
input_dir = '../data/set_04/train_01'  # Thay bằng đường dẫn thật nếu cần
output_dir = '../data/set_04/train_01/merged_no_accessory'   # Thư mục đầu ra
os.makedirs(output_dir, exist_ok=True)

# Danh sách các nhóm attack chính
main_attack_types = ['rpm', 'speed', 'interval', 'standstill', 'systematic', 'attack-free']

# Gom file theo nhóm attack chính
grouped_files = defaultdict(list)

# Duyệt toàn bộ file
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        # Bỏ qua các file accessory
        if 'accessory' in file_name.lower():
            print(f"⚠️ Bỏ qua file: {file_name}")
            continue  # Skip accessory files

        # Tìm attack type chính phù hợp với tên file
        matched = False
        for atk in main_attack_types:
            if atk in file_name.lower():
                grouped_files[atk].append(file_name)
                matched = True
                break
        if not matched:
            print(f"⚠️ Không xác định được nhóm cho file: {file_name}")

# Duyệt từng nhóm để merge
for group, file_list in grouped_files.items():
    df_list = []
    print(f"\n📂 Đang gộp nhóm: {group}")

    for file_name in file_list:
        file_path = os.path.join(input_dir, file_name)
        try:
            df = pd.read_csv(file_path)

            # Xử lý arbitration_id
            if 'arbitration_id' in df.columns:
                df['arbitration_id'] = df['arbitration_id'].apply(
                    lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
                )

            # Xử lý data_field
            if 'data_field' in df.columns:
                df['data_field'] = df['data_field'].apply(
                    lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
                )

            # Lấy cột chính nếu có
            keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
            df = df[[col for col in keep_cols if col in df.columns]].dropna()

            df_list.append(df)
        except Exception as e:
            print(f"[!] Lỗi đọc file {file_name}: {e}")

    # Gộp và lưu cho các nhóm khác ngoài 'attack-free'
    if group != 'attack-free':
        if df_list:
            merged_df = pd.concat(df_list, ignore_index=True)
            output_file = os.path.join(output_dir, f'{group}.csv')
            merged_df.to_csv(output_file, index=False)
            print(f"✅ Đã lưu file: {output_file}")
        else:
            print(f"⚠️ Không có file hợp lệ trong nhóm {group}")

# Chia dữ liệu của 'attack-free' vào các nhóm attack khác
attack_free_df_list = []

# Đọc tất cả các tệp 'attack-free'
for file_name in grouped_files['attack-free']:
    file_path = os.path.join(input_dir, file_name)
    try:
        df = pd.read_csv(file_path)

        # Xử lý dữ liệu tương tự như trên
        if 'arbitration_id' in df.columns:
            df['arbitration_id'] = df['arbitration_id'].apply(
                lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
            )

        if 'data_field' in df.columns:
            df['data_field'] = df['data_field'].apply(
                lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
            )

        keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
        df = df[[col for col in keep_cols if col in df.columns]].dropna()

        attack_free_df_list.append(df)
    except Exception as e:
        print(f"[!] Lỗi đọc file {file_name}: {e}")

# Gộp tất cả dữ liệu 'attack-free'
attack_free_df = pd.concat(attack_free_df_list, ignore_index=True)

# Chia đều dữ liệu 'attack-free' vào các nhóm attack chính
attack_free_samples_per_group = len(attack_free_df) // len(main_attack_types[:-1])  # Không tính 'attack-free' ở đây
attack_free_split = np.array_split(attack_free_df, len(main_attack_types)-1)

# Lưu kết quả và ghi đè lên các file tấn công hiện tại
for idx, group in enumerate(main_attack_types[:-1]):  # Bỏ qua 'attack-free'
    group_file_path = os.path.join(output_dir, f'{group}.csv')
    attack_free_data = attack_free_split[idx]
    
    # Đọc dữ liệu nhóm tấn công hiện tại
    group_data = pd.read_csv(group_file_path)
    
    # Gộp dữ liệu nhóm hiện tại với dữ liệu 'attack-free' chia đều
    merged_data = pd.concat([group_data, attack_free_data], ignore_index=True)
    
    # Ghi đè lên file tấn công hiện tại
    merged_data.to_csv(group_file_path, index=False)
    print(f"✅ Đã ghi đè file: {group_file_path}")

print("DONE!")


⚠️ Bỏ qua file: rpm-accessory-4.csv
⚠️ Bỏ qua file: speed-accessory-3.csv
⚠️ Bỏ qua file: accessory-3.csv
⚠️ Bỏ qua file: accessory-4.csv
⚠️ Bỏ qua file: speed-accessory-4.csv
⚠️ Bỏ qua file: rpm-accessory-3.csv

📂 Đang gộp nhóm: attack-free

📂 Đang gộp nhóm: standstill
✅ Đã lưu file: ../data/set_04/train_01/merged_no_accessory/standstill.csv

📂 Đang gộp nhóm: systematic
✅ Đã lưu file: ../data/set_04/train_01/merged_no_accessory/systematic.csv

📂 Đang gộp nhóm: speed
✅ Đã lưu file: ../data/set_04/train_01/merged_no_accessory/speed.csv

📂 Đang gộp nhóm: interval
✅ Đã lưu file: ../data/set_04/train_01/merged_no_accessory/interval.csv

📂 Đang gộp nhóm: rpm
✅ Đã lưu file: ../data/set_04/train_01/merged_no_accessory/rpm.csv


/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


✅ Đã ghi đè file: ../data/set_04/train_01/merged_no_accessory/rpm.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged_no_accessory/speed.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged_no_accessory/interval.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged_no_accessory/standstill.csv
✅ Đã ghi đè file: ../data/set_04/train_01/merged_no_accessory/systematic.csv
DONE!


In [4]:
import os
import pandas as pd
from collections import defaultdict

# Đường dẫn thư mục chứa các file attack
input_dir = '../data/set_04/test_01_known_vehicle_known_attack'
output_dir = '../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory'   # Thư mục đầu ra
os.makedirs(output_dir, exist_ok=True)

# Danh sách các nhóm attack chính
# main_attack_types = ['dos', 'double', 'force-neutral', 'fuzzing', 'triple']
main_attack_types = ['rpm', 'speed', 'interval', 'standstill', 'systematic', 'attack-free']

# Gom file theo nhóm attack chính
grouped_files = defaultdict(list)

# Duyệt toàn bộ file
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        # Bỏ qua các file accessory
        if 'accessory' in file_name.lower():
            print(f"⚠️ Bỏ qua file: {file_name}")
            continue  # Skip accessory files

        # Tìm attack type chính phù hợp với tên file
        matched = False
        for atk in main_attack_types:
            if atk in file_name.lower():
                grouped_files[atk].append(file_name)
                matched = True
                break
        if not matched:
            print(f"⚠️ Không xác định được nhóm cho file: {file_name}")

# Duyệt từng nhóm để merge
for group, file_list in grouped_files.items():
    df_list = []
    print(f"\n📂 Đang gộp nhóm: {group}")

    for file_name in file_list:
        file_path = os.path.join(input_dir, file_name)
        try:
            df = pd.read_csv(file_path)

            # Xử lý arbitration_id
            if 'arbitration_id' in df.columns:
                df['arbitration_id'] = df['arbitration_id'].apply(
                    lambda x: int(x, 16) if isinstance(x, str) and (x.startswith('0x') or all(c in '0123456789ABCDEF' for c in x)) else int(x)
                )

            # Xử lý data_field
            if 'data_field' in df.columns:
                df['data_field'] = df['data_field'].apply(
                    lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x
                )

            # Lấy cột chính nếu có
            keep_cols = ['timestamp', 'arbitration_id', 'data_field', 'attack']
            df = df[[col for col in keep_cols if col in df.columns]].dropna()

            df_list.append(df)
        except Exception as e:
            print(f"[!] Lỗi đọc file {file_name}: {e}")

    # Gộp và lưu
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        output_file = os.path.join(output_dir, f'{group}.csv')
        merged_df.to_csv(output_file, index=False)
        print(f"✅ Đã lưu file: {output_file}")
    else:
        print(f"⚠️ Không có file hợp lệ trong nhóm {group}")

print("DONE!")


⚠️ Bỏ qua file: speed-accessory-1.csv
⚠️ Bỏ qua file: rpm-accessory-2.csv
⚠️ Bỏ qua file: speed-accessory-2.csv
⚠️ Bỏ qua file: rpm-accessory-1.csv

📂 Đang gộp nhóm: interval
✅ Đã lưu file: ../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory/interval.csv

📂 Đang gộp nhóm: systematic
✅ Đã lưu file: ../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory/systematic.csv

📂 Đang gộp nhóm: rpm
✅ Đã lưu file: ../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory/rpm.csv

📂 Đang gộp nhóm: speed
✅ Đã lưu file: ../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory/speed.csv

📂 Đang gộp nhóm: standstill
✅ Đã lưu file: ../data/set_04/test_01_known_vehicle_known_attack/merged_no_accessory/standstill.csv
DONE!


In [10]:
# Define the directory path
directory_path = '../data/set_04/train_01/merged/interval.csv'
df = pd.read_csv(directory_path)
# Display the first few rows of the combined DataFrame
filtered_df = df[df['attack'] != 0]
filtered_df

,timestamp,arbitration_id,data_field,attack
118815,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
118816,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
118817,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
118818,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
118852,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
...,...,...,...,...
779147,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
779182,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
779215,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1
779246,1.672531e+09,212,0011010100000001001101000000000100110100000000...,1


In [1]:
import os
import pandas as pd

# Directory path containing CSV files
# /home/hieutt/UniCon/data/can-ml/2017-subaru-forester/merged
input_dir = '../data/can-ml/2017-subaru-forester/merged'  # Change the path if needed

# List of CSV files in the directory
files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# Function to check each file
def check_data(file_path):
    print(f"\nChecking file: {file_path}")
    
    # Read the file
    df = pd.read_csv(file_path)
    
    # Display the shape of the DataFrame
    print("Shape:", df.shape)
    
    # Display the first 5 rows
    print("Head:")
    display(df.head())
    
    # Check for NaN values
    nan_count = df.isna().sum().sum()
    if nan_count > 0:
        print("Warning: There are NaN values in the data.")
        print(df.isna().sum())
    else:
        print("No NaN values found.")
    
    # Check data types
    print("\nData types:")
    print(df.dtypes)
    
    # Check label distribution if 'label' column exists
    if 'label' in df.columns:
        print("\nLabel distribution:")
        print(df['label'].value_counts())
    
    # Descriptive statistics for numeric columns
    print("\nDescriptive statistics for numeric columns:")
    print(df.describe())
    
# Iterate over each file and check
for file in files:
    check_data(os.path.join(input_dir, file))



Checking file: ../data/can-ml/2017-subaru-forester/merged/combined.csv
Shape: (5806825, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,356,0000001000000000000000000000000000000000000000...,0
1,1.672531e+09,881,0000000000000000000000001100000011111110000111...,0
2,1.672531e+09,328,0000101100010100010100001000001001000001000000...,0
3,1.672531e+09,329,0111111000000001001111111001100110101010001100...,0
4,1.672531e+09,117,0000000000000000000000000000000000000000000010...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  5.806825e+06    5.806825e+06  5.806825e+06
mean   1.672531e+09    4.111147e+02  1.039036e-02
std    1.636970e+02    3.333655e+02  1.014022e-01
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/DoS.csv
Shape: (2297897, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,342,0000000110000000000000000000000000000000000000...,0
1,1.672531e+09,320,0000000000000100101010000000100111010010110100...,0
2,1.672531e+09,321,0011101000110000100001110010111110101000110010...,0
3,1.672531e+09,864,0100000000010100100010011000001100110001011100...,0
4,1.672531e+09,865,0000000000111010000000001101010000001000000000...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.297897e+06    2.297897e+06  2.297897e+06
mean   1.672532e+09    4.054529e+02  2.254148e-02
std    2.930624e+02    3.377873e+02  1.484364e-01
min    1.672531e+09    0.000000e+00  0.000000e+00
25%    1.672531e+09    2.100000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/speed.csv
Shape: (3672497, 4)
Head:


...

No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.623432e+06    2.623432e+06  2.623432e+06
mean   1.672531e+09    4.051509e+02  4.052478e-02
std    1.451882e+02    3.309302e+02  1.971865e-01
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/systematic.csv
Shape: (3397184, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,320,0000000000000001001001000000100000101000001010...,0
1,1.672531e+09,321,1110001100101010001111110010101100100100100010...,0
2,1.672531e+09,324,0000000000000011011101110011100000110000101010...,0
3,1.672531e+09,328,0000101100010000001000011000100001000001000000...,0
4,1.672531e+09,329,0001011000001100001111110010100110101010000111...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  3.397184e+06    3.397184e+06  3.397184e+06
mean   1.672531e+09    4.165107e+02  6.579567e-03
std    1.962385e+02    3.403088e+02  8.084725e-02
min    1.672531e+09    0.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    2.047000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/standstill.csv
Shape: (2361472, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,881,0000000100000000000000010000000000000110101000...,0
1,1.672531e+09,328,0000101100010000101010011000011001000001000000...,0
2,1.672531e+09,329,0001011000001100001111110010001010101001000110...,0
3,1.672531e+09,320,0001101000001001101010010000011000000000000110...,0
4,1.672531e+09,321,1000000100101001101110100010100010101001000001...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.361472e+06    2.361472e+06  2.361472e+06
mean   1.672531e+09    4.128220e+02  4.576806e-03
std    1.523671e+02    3.345799e+02  6.749712e-02
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00
